# 일단 이 파일을 본인의 디렉토리로 옮긴 후에 실행바랍니다.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import timm
from sklearn.metrics import f1_score
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from sklearn.model_selection import train_test_split
import timeit
from tqdm import tqdm
import random
import math
from efficientnet_pytorch import EfficientNet
from adamp import AdamP

# 이 부분은 정말로 주의가 필요합니다
### 해당 코드는 정말로 딱 한번만 실행하세요. 2번도 안됩니다.. 딱 한번이요 한번 실행하고 나면 주석처리하거나 삭제해주세요~
### 왜냐하면, 실제 이미지 파일에서 마스크 이상하게 쓴 사람과 안 쓴 사람이 서로 바뀌어져있는데,
### 잘못된 이미지 파일이기 때문에 바꾸는 부분입니다.
### 한번 실행하고 또 실행하면 또 바뀌니,, 다시 이상한 데이터가 되니깐 꼭 한번만 실행해주세요

In [1]:
# data_dir= "/opt/ml/input/data/train/images"

# wrong_folders = ["000020_female_Asian_50", "004418_male_Asian_20", "005227_male_Asian_22"]

# file_names = ["incorrect_mask.jpg", "normal.jpg", "temp.jpg"]

# for folder in wrong_folders:

#     image_dir = os.path.join(data_dir, folder)

#     incorrect_file = os.path.join(image_dir, file_names[0])

#     normal_file = os.path.join(image_dir, file_names[1])

#     temp = os.path.join(image_dir, file_names[2])

#     os.rename(incorrect_file, temp)  

#     os.rename(normal_file, incorrect_file)

#     os.rename(temp, normal_file)

#     print("Changed File Names")

## 이 부분은 사용할 데이터 처리하는 부분입니다.
## 한번 돌리는데 시간이 조금 걸리는 편이니, 아래에 실행을 해서 데이터프레임이 만들어진다면,
## 그냥 불러서 사용하세요

In [16]:
train_dir = '/opt/ml/input/data/train/images'
test_dir = '/opt/ml/input/data/eval/'

train_df = pd.read_csv("/opt/ml/input/data/train/train.csv")

train_df.loc[train_df['id']=='001498-1','gender'] = 'female'
train_df.loc[train_df['id']=='004432','gender'] = 'female'
train_df.loc[train_df['id']=='000010','gender'] = 'male'
train_df.loc[train_df['id']=='000357','gender'] = 'male'
train_df.loc[train_df['id']=='000664','gender'] = 'male'
train_df.loc[train_df['id']=='000667','gender'] = 'male'
train_df.loc[train_df['id']=='000725','gender'] = 'male'
train_df.loc[train_df['id']=='000736','gender'] = 'male'
train_df.loc[train_df['id']=='000767','gender'] = 'male'
train_df.loc[train_df['id']=='000817','gender'] = 'male'
train_df.loc[train_df['id']=='003780','gender'] = 'male'
train_df.loc[train_df['id']=='003798','gender'] = 'male'
train_df.loc[train_df['id']=='004281','gender'] = 'male'
train_df.loc[train_df['id']=='006359','gender'] = 'male'
train_df.loc[train_df['id']=='006360','gender'] = 'male'
train_df.loc[train_df['id']=='006361','gender'] = 'male'
train_df.loc[train_df['id']=='006362','gender'] = 'male'
train_df.loc[train_df['id']=='006363','gender'] = 'male'
train_df.loc[train_df['id']=='006364','gender'] = 'male'
train_df.loc[train_df['id']=='006504','gender'] = 'male'
train_df.loc[train_df['id']=='001009','age'] = 29
train_df.loc[train_df['id']=='001064','age'] = 29
train_df.loc[train_df['id']=='001637','age'] = 29
train_df.loc[train_df['id']=='001666','age'] = 29
train_df.loc[train_df['id']=='001852','age'] = 29
train_df.loc[train_df['id']=='004348','age'] = 60

def age_group(x):
    if x < 30:
        return 0
    elif x < 58:
        return 1
    else:
        return 2
    
def gender_group(x):
    if x == 'male':
        return 0
    else:
        return 1
    
df = pd.DataFrame(None, columns = ['gender', 'age','maskOX','maskGB','label','path'])

for index, line in enumerate(train_df.iloc):
    for file in list(os.listdir(os.path.join(train_dir, line['path']))):
        if file[0] == '.':
            continue
        if file.split('.')[0] == 'normal':
            mask = 2
        elif file.split('.')[0] == 'incorrect_mask':
            mask = 1
        else:
            mask = 0
        gender = 0 if line['gender'] == 'male' else 1
        data = {
            'gender' : gender_group(line['gender']),
            'age' : age_group(line['age']),
            'maskOX' : 0 if mask == 2 else 1, # 마스크 안쓰면 0, 쓰면 1
            'maskGB' : None if mask == 2 else (0 if mask == 1 else 1), # 마스크 안쓰면 none, 마스크 비정상이면 0, 마스크 정상이면 1
            'path': os.path.join(train_dir, line['path'], file),
            'label': mask * 6 + gender * 3 + age_group(line['age'])
        }
        df = df.append(data, ignore_index=True)
# 마스크 5:1:1
df.to_csv('/opt/ml/code/total.csv', index=False)

df = pd.DataFrame(None, columns = ['gender', 'age','maskOX','maskGB','label','path'])

for index, line in enumerate(train_df.iloc):
    for file in list(os.listdir(os.path.join(train_dir, line['path']))):
        if file[0] == '.':
            continue
        if file.split('.')[0] == 'normal':
            mask = 2
        elif file.split('.')[0] == 'incorrect_mask':
            mask = 1
        elif file.split('.')[0] == 'mask1':
            mask = 0
        else:
            continue
        gender = 0 if line['gender'] == 'male' else 1
        data = {
            'gender' : gender_group(line['gender']),
            'age' : age_group(line['age']),
            'maskOX' : 0 if mask == 2 else 1, # 마스크 안쓰면 0, 쓰면 1
            'maskGB' : None if mask == 2 else (0 if mask == 1 else 1), # 마스크 안쓰면 none, 마스크 비정상이면 0, 마스크 정상이면 1
            'path': os.path.join(train_dir, line['path'], file),
            'label': mask * 6 + gender * 3 + age_group(line['age'])
        }
        df = df.append(data, ignore_index=True)
# 마스크 1:1:1
df.to_csv('/opt/ml/code/total_111.csv', index=False)


# 시드 및 모델의 기본적인 파라미터 고정

In [2]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

# GPU 확인 및 device에 사용할 gpu 저장

In [3]:
print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.6.0
GPU 사용 가능 여부: True


# 모델 돌릴 조건 나누는 곳

In [4]:
total = pd.read_csv("/opt/ml/code/total.csv")

# 1대1대1 비율로 돌리고 싶다면 아래 코드를 쓰세요
# total = pd.read_csv("/opt/ml/code/total111.csv")

In [5]:
def change_df_condition(df, num):
    # 전체 데이터 활용
    if num == 0:
        return df
    # 마스크 쓴 사람
    elif num == 1:
        temp_df = df.loc[df['maskOX']==1]
        return temp_df
    # 마스크 안쓴 사람
    elif num == 2:
        temp_df = df.loc[df['maskOX']==0]
        return temp_df

In [6]:
def output(target):
    if target == "label":
        return 18
    elif target == "gender":
        return 2
    elif target == "age":
        return 3
    elif target == "maskOX":
        return 2
    elif target == "maskGB":
        return 2

In [7]:
# 아래와 같은 조건이면, 전체 데이터 중에서 마스크 쓴 사람과 안 쓴사람을 목표로 한다
df = change_df_condition(total, 0)
target ="label"
CLASS_NUM = output(target)

In [8]:
train, valid = train_test_split(df, test_size=0.2,
                                shuffle=True, stratify=df[target],
                                random_state=42)

# 데이터셋 만들기

In [10]:
class CustomDataset(Dataset): # 각 목표별 데이터셋 만들기
    def __init__(self, df, transform, target):
        self.df = df
        self.img_paths = self.df['path'].tolist()
        self.labels = self.df[target].tolist()
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(self.labels[index])

    def __len__(self):
        return len(self.img_paths)

## transform 정의 -> 일단은 고정하고 추후에 바꿔보자

In [11]:
transform = {
    'train': transforms.Compose([
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [12]:
BATCH_SIZE = 32

train_dataset = CustomDataset(train, transform['train'], target)

train_dataloader = DataLoader(
                            train_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=True
                             )

valid_dataset = CustomDataset(valid, transform['test'], target)

valid_dataloader = DataLoader(
                            valid_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=False
                             )

# 모델생성

In [13]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):

        ce_loss = F.cross_entropy(input, target,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [16]:
model = EfficientNet.from_pretrained('efficientnet-b5', num_classes=18)
model.to(device)

optimizer = AdamP(model.parameters(),lr=5e-4,weight_decay=5e-4)
criterion = FocalLoss()

# 여기서 각자 저장할 모델 지정해주세요
# 모델 저장해야지 중간에 종료해도 그 결과 남아서 써먹을수있어요~ 중간에 멈췄다가 다시 로드해서 재학습도 쌉가능
saved_dir = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/model'
val_every = 1

Loaded pretrained weights for efficientnet-b5


# 학습 및 평가

In [17]:
def func_eval(model,data_iter,device):
    with torch.no_grad():
        n_total,n_correct = 0,0
        model.eval() # evaluate (affects DropOut and BN)
        for batch_in,batch_out in data_iter:
            y_trgt = batch_out.to(device)
            model_pred = model(batch_in.to(device))
            _,y_pred = torch.max(model_pred.data,1)
            n_correct += (y_pred==y_trgt).sum().item()
            n_total += batch_in.size(0)
        val_accr = (n_correct/n_total)
        model.train() # back to train mode 
    return val_accr

In [18]:
def train(num_epochs, model, data_loader, criterion, optimizer, saved_dir, val_every, device):
    model.train()
    print('Start training..')
    total_start_time = timeit.default_timer()
    best_loss = 9999999
    best_test_accuracy = 0
    for epoch in tqdm(range(num_epochs)):
        epoch_f1 = 0
        running_acc = 0
        epoch_loss = 0
        print('Epoch start..')
        epoch_start_time = timeit.default_timer()
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()         
            loss.backward()
            optimizer.step()

            _, argmax = torch.max(outputs, 1)
            accuracy = (labels == argmax).float().mean()
            
            f1 = f1_score(labels.detach().cpu().numpy(), argmax.detach().cpu().numpy(), average='weighted')
            epoch_f1 += f1
            running_acc += accuracy
            epoch_loss += loss.item()
            if (i+1) % 3 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.10f}, Accuracy: {:.2f}%, F1_Score: {:.2f}'.format(
                    epoch+1, num_epochs, i+1, len(data_loader), loss.item(), accuracy.item() * 100, f1))
        print("------------Epoch Finish------------")
        avrg_epoch_loss = epoch_loss/(i+1)
        train_accr = func_eval(model,data_loader,device)
        print('Epoch [{}/{}], Avrg Accuracy: {:.2f}%, Avrg Loss: {:.10f}, Train Accuracy: {:.2f}%, F1_Score: {:.2f}'.format(
                    epoch+1, num_epochs, running_acc.item()/(i+1) * 100, avrg_epoch_loss,train_accr * 100, epoch_f1/(i+1)))
        if (epoch + 1) % val_every == 0:
            avrg_val_loss = validation(epoch + 1, model, valid_dataloader, criterion, device)
            print("avrg val loss : {:.10f}".format(avrg_val_loss))
            if avrg_val_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_val_loss
                save_model(model, saved_dir,epoch)
        epoch_end_time = timeit.default_timer()
        print("Epoch end..")
        print(f"epoch time : {epoch_end_time-epoch_start_time}")
        epoch_acc = running_acc / (i+1)
        
        if best_test_accuracy < epoch_acc:
            best_test_accuracy = epoch_acc
            save_model(model, saved_dir,epoch)
            early_stop_point = 0
        else:
            early_stop_point += 1
        if early_stop_point == 3:
            print('early_stopped')
            break
    print('End training..')
    total_end_time = timeit.default_timer()
    print(f"total time : {total_end_time-total_start_time}")

In [19]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        epoch_f1 = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            ## 코드 종료 ##
            total += imgs.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item()
            total_loss += loss
            cnt += 1
            epoch_f1 += f1_score(labels.detach().cpu().numpy(), argmax.detach().cpu().numpy(), average='weighted')
        avrg_loss = total_loss / cnt
        avrg_f1 = epoch_f1 / cnt
        print('Validation #{}  Accuracy: {:.2f}% F1_Score: {:.2f} Average Loss: {:.10f}'.format(epoch, correct / total * 100,avrg_f1 ,avrg_loss))
    model.train()
    return avrg_loss

# 모델 저장 하는 함수
### 여기서 각자 저장할 모델이름을 변경해주세요~ 조건이나 날짜 시간 등을 추가하면 구분하기 쉽겠죠?
### 매번 새로 돌릴때마다 꼭 변경해서 기록해야지 나중에 확인하기 편합니다
### 여기 수정할 때, 위에 모델들의 조건을 정리해서 노션에 올리면 더 좋겠죠??

In [20]:
def save_model(model, saved_dir,epoch):
    os.makedirs(saved_dir, exist_ok=True)
    file_name=f'effi_b5_epoch{epoch}.pt'
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point,output_path)

### 학습 시작

In [21]:
num_epochs = 20
train(num_epochs, model, train_dataloader, criterion, optimizer, saved_dir, val_every, device)

  0%|          | 0/20 [00:00<?, ?it/s]

Start training..
Epoch start..
Epoch [1/20], Step [3/473], Loss: 2.4543766975, Accuracy: 25.00%, F1_Score: 0.22


  0%|          | 0/20 [00:05<?, ?it/s]


KeyboardInterrupt: 

# 모델 불러오는 곳
### 어제 돌리다가 일어나서 돌리던 모델 다시 돌리고 싶으면. 아래에서 모델 불러와서 쓰면 됩니다.
### 대신 완전히 껐다가 킨거면, 위에 train 함수 전까지 다 실행시키고 아래 코드 실행하고 다시 train 학습 하면 됩니다.

In [22]:
model_path = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/model/effi_b5_epoch2.pt'
checkpoint = torch.load(model_path,map_location=device)
state_dict = checkpoint['net']
model.load_state_dict(state_dict)

<All keys matched successfully>

# 틀린 부분 확인하는 곳

In [36]:
def check_eval(raw_data, dataloader, model, device):
    result = []
    with torch.no_grad():
        model.eval()
        for i, (X,y) in enumerate(dataloader):
            model_pred = model.forward(X.to(device))
            _, y_pred = torch.max(model_pred, 1)
            
            result.append([valid.iloc[i]['path'], y_pred.cpu().numpy()[0], y.cpu().numpy()[0]])
    result = pd.DataFrame(result, columns=['path', 'pred', 'target'])
    return result

In [37]:
valid_testing_dataloader = DataLoader(valid_dataset, shuffle = False)
check_eval_df = check_eval(valid, valid_testing_dataloader, model, device)

In [38]:
wrong_df = check_eval_df[check_eval_df['pred'] != check_eval_df['target']]
wrong_df = wrong_df.reset_index(drop=True)

In [40]:
len(wrong_df)

<Figure size 3600x7200 with 0 Axes>

# 아래는 실제 제출하기 위한 공간입니다
## class(18)가 아니면 안쓰면 됩니다.

In [32]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [41]:
test_dir = '/opt/ml/input/data/eval/'

submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

test_data = TestDataset(image_paths, transform=transform['test'])
test_loader = DataLoader(test_data, shuffle=False)

In [43]:
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
model.eval()
i = 0
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        _, predict = torch.max(pred, 1)
        if i < 3:
            i+=1
            print(predict)
        all_predictions.extend(predict.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submit_dir = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/submit'
submission.to_csv(os.path.join(submit_dir, 'effi_2.csv'), index=False)
print('test inference is done!')

tensor([13], device='cuda:0')
tensor([1], device='cuda:0')
tensor([13], device='cuda:0')
test inference is done!


## 여기서 i를 바꿔가면서 확인해보자
## 확인하면서 깃허브에 있는 info.csv를 채워보는건 어떨까?

In [ ]:
i = 209
a = submission['ans'][i] // 6
b = submission['ans'][i] % 6
c = submission['ans'][i] % 3
print(submission['ans'][i])
if a == 0:
    print("마스크씀")
elif a == 1:
    print("마스크 비정상")
else:
    print("마스크 안씀")
if b < 3:
    print("남자")
else:
    print("여자")
if c == 0 :
    print("젊음")
elif c == 1:
    print("중년")
else:
    print("노년")
temp_dir = os.path.join(image_dir, submission['ImageID'][i])
print(temp_dir)
Image.open(temp_dir)

In [ ]:
submission2 = pd.read_csv(os.path.join('/opt/ml/level1-image-classification-level1-recsys-16/', 'info.csv'))
f1_score(submission2.iloc[:200]['ans'], submission.iloc[:200]['ans'], average='macro')